In [2]:
import pandas as pd 

In [ ]:
class RMvisualiser():


    def load_data(self,path):
        #not sure how consistant is the data format between readouts
        self.data = pd.read_excel(path,usecols="G:AG")

    def get_full(self):
        return self.data.to_numpy()

    def get_all(self):
        return self.data.to_numpy()[3:,1:]
    
    def get_all_wavelenght(self, wavelength):
        
        temp = self.data.to_numpy()[3:,0] == wavelength

        print(temp)
        return self.data.to_numpy()[3:,0:][temp][:,1:]


    def select_n_connectors(self,indecies):



        temp = self.get_all()[:,indecies]

        print(temp)
        return
    


In [16]:
test = RMvisualiser()

test.load_data("RM example data.xlsx")


test.get_full()

array([[nan, nan, nan, ..., nan, nan, nan],
       ['Wavelenght\n[nm]', 1.0, nan, ..., nan, 13.0, nan],
       [nan, 1.0, 2.0, ..., 24.0, 25.0, 26.0],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=object)

In [17]:
test.select_n_connectors([0,1,2])

[[nan nan 0.01]
 [nan nan 0.03]
 [nan nan 0.04]
 [nan nan 0.02]
 [nan nan 0.03]
 [nan nan 0.03]
 [0.02 0.0 nan]
 [0.04 0.01 nan]
 [0.04 0.01 nan]
 [0.05 0.06 nan]
 [0.05 0.04 nan]
 [0.04 0.02 nan]
 [0.11 0.08 0.2]
 [0.11 0.08 0.2]
 [0.1 0.07 0.17]
 [0.18 0.03 0.36]
 [0.15 0.04 0.31]
 [0.13 0.04 0.32]
 [0.05 0.09 0.04]
 [0.08 0.14 0.05]
 [0.11 0.23 0.05]
 [0.01 0.05 0.01]
 [0.01 0.06 0.02]
 [0.0 0.02 0.02]
 [0.01 0.06 0.12]
 [0.05 0.07 0.14]
 [0.12 0.11 0.12]
 [0.04 0.11 0.09]
 [0.05 0.09 0.1]
 [0.02 0.06 0.12]
 [0.17 0.03 0.11]
 [0.15 0.03 0.12]
 [0.22 0.07 0.08]
 [0.07 0.04 0.08]
 [0.06 0.03 0.1]
 [0.02 0.01 0.11]
 [0.06 0.06 0.1]
 [0.07 0.07 0.11]
 [0.18 0.1 0.08]
 [0.04 0.01 0.05]
 [0.02 0.01 0.06]
 [0.01 0.0 0.12]
 [0.04 0.01 0.02]
 [0.03 0.01 0.03]
 [0.05 0.01 0.04]
 [0.03 0.02 0.01]
 [0.01 0.02 0.02]
 [0.0 0.0 0.02]
 [0.07 0.02 0.04]
 [0.07 0.02 0.05]
 [0.11 0.03 0.05]
 [0.03 0.07 0.04]
 [0.02 0.06 0.05]
 [0.0 0.03 0.09]
 [0.08 0.05 0.1]
 [0.05 0.05 0.07]
 [0.05 0.04 0.07]
 [0.13

In [8]:
pd.read_excel("RM example data.xlsx",
              usecols="G:AG")

,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,...,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Wavelenght\n[nm],1.0,NaN,2.00,NaN,3.00,NaN,4.00,NaN,5.00,...,9.00,NaN,10.00,NaN,11.00,NaN,12.00,NaN,13.00,NaN
2,NaN,1.0,2.0,3.00,4.00,5.00,6.00,7.00,8.00,9.00,...,17.00,18.00,19.00,20.00,21.00,22.00,23.00,24.00,25.00,26.00
3,1310,NaN,NaN,0.01,0.04,0.15,0.26,0.03,0.04,0.08,...,0.05,0.04,0.15,0.18,0.06,0.04,0.03,0.01,0.06,0.01
4,1550,NaN,NaN,0.03,0.03,0.13,0.21,0.03,0.02,0.09,...,0.04,0.04,0.13,0.17,0.06,0.04,0.04,0.02,0.04,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
